<a href="https://colab.research.google.com/github/superoverflow/Notebooks/blob/main/random_num_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [167]:
import plotly.express as px
import numpy as np
from functools import reduce
from random import random
from tqdm import tqdm

def random_but_greater_than(previous_value: float, bottom_line: float):
  new_value = previous_value + np.random.randn()
  return max(new_value, bottom_line)

def generate_data(size: float):
  x = np.arange(0, size)
  m = random() * 0.05
  c = np.random.randn()
  min_y = m * x + c
  y = reduce(lambda a, b: [*a, max(a[-1] + np.random.randn(), m * b + c)], x, [0])
  y = np.delete(y, 0)
  return [x, y, m, c]


In [168]:
SIZE = 1024
data = []

for i in tqdm(range(100_000)):
  data.append(generate_data(1024))

100%|██████████| 100000/100000 [10:11<00:00, 163.50it/s]


In [169]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

rows = 5
cols = 5

fig = make_subplots(rows=rows, cols=cols)

for i in range(rows):
  for j in range(cols):
    x, y, m, c = data[i + j]
    fig.append_trace(go.Scatter(x=x, y=y,), row=i+1, col=j+1)
    fig.append_trace(go.Line(x=x, y=(m * x + c)), row=i+1, col=j+1)

fig.update_layout(height=800, width=1200, showlegend=False)
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [170]:
def scale_data(x, y, m, c):
  min_ = min(y)
  scale_= max(y) - min_
  y_head = (y - min_) / scale_

  m_head = m / scale_
  c_head = (c - min_) / scale_
  return x, y_head, m_head, c_head


data_head = [scale_data(item[0], item[1], item[2], item[3]) for item in data]

In [171]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

rows = 5
cols = 5

fig = make_subplots(rows=rows, cols=cols)

for i in range(rows):
  for j in range(cols):
    x, y, m, c = data_head[i + j]
    fig.append_trace(go.Scatter(x=x, y=y,), row=i+1, col=j+1)
    fig.append_trace(go.Line(x=x, y=(m * x + c)), row=i+1, col=j+1)

fig.update_layout(height=800, width=1200, showlegend=False)
fig.show()

In [172]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

X = list(item[1] for item in data_head)
y = list(item[2] for item in data_head)

X_train, X_test, y_train, y_test = train_test_split(X, y)
regr = linear_model.Ridge(alpha=.5)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)


In [173]:
fig = make_subplots(rows=rows, cols=cols)

for i in range(rows):
  for j in range(cols):
    x = np.arange(0, 1024)
    y = X_test[50 + i + j]
    m = y_pred[50 +i + j]

    fig.append_trace(go.Scatter(x=x, y=y), row=i+1, col=j+1)
    fig.append_trace(go.Line(x=x, y=(m * x)), row=i+1, col=j+1)

fig.update_layout(height=800, width=1200, showlegend=False)
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


